In [6]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision
from torchvision import transforms, datasets, models
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
from torch.utils.data import DataLoader, Dataset

# Verificar si se dispone de una GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Configuración de hiperparámetros
EPOCHS = 50
BATCH_SIZE = 64
LEARNING_RATE = 0.001
IMAGE_SIZE = (128, 128)
ALPHA = 0.2  # Parámetro para Mixup

# Directorio del dataset
DATASET_DIR = '../data/arcgis-survey-images-new'


Using device: cuda


In [7]:

# Dataset personalizado para aplicar Albumentations y conversiones necesarias
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = plt.imread(self.image_paths[idx])
        label = self.labels[idx]
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented["image"]
        return image, label

# Definir la aumentación avanzada con Albumentations
train_transform = A.Compose([
    A.Resize(IMAGE_SIZE[0], IMAGE_SIZE[1]),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.GaussNoise(p=0.2),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(IMAGE_SIZE[0], IMAGE_SIZE[1]),
    ToTensorV2()
])


In [10]:
import os
import shutil
from glob import glob
from sklearn.model_selection import train_test_split

# Definir función para crear las carpetas y mover las imágenes
def create_dataset_splits(data_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, seed=123):
    # Verificar que la suma de los ratios sea 1
    assert train_ratio + val_ratio + test_ratio == 1, "La suma de train, val y test ratios debe ser 1."
    
    # Crear carpetas para los splits
    for split in ["train", "val", "test"]:
        split_dir = os.path.join(output_dir, split)
        if not os.path.exists(split_dir):
            os.makedirs(split_dir)

    # Iterar sobre cada clase en el directorio original
    for class_name in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_dir):
            continue

        # Crear subcarpetas para cada clase en train, val y test
        for split in ["train", "val", "test"]:
            split_class_dir = os.path.join(output_dir, split, class_name)
            if not os.path.exists(split_class_dir):
                os.makedirs(split_class_dir)

        # Obtener las rutas de las imágenes para la clase actual
        image_paths = glob(os.path.join(class_dir, "*.jpg"))  # Cambiar extensión si es necesario

        # Separar el dataset en train, val y test
        train_paths, temp_paths = train_test_split(image_paths, train_size=train_ratio, random_state=seed)
        val_paths, test_paths = train_test_split(temp_paths, test_size=test_ratio/(val_ratio + test_ratio), random_state=seed)

        # Mover las imágenes a las carpetas correspondientes
        for split, paths in zip(["train", "val", "test"], [train_paths, val_paths, test_paths]):
            for img_path in paths:
                shutil.copy(img_path, os.path.join(output_dir, split, class_name, os.path.basename(img_path)))

# Ruta del dataset original y la salida
original_dataset_dir = '../data/arcgis-survey-images-new'  # Cambiar a la ruta del directorio original de las imágenes
output_dataset_dir = '../data/arcgis-survey-images-new/split'  # Cambiar a la ruta donde se crearán los splits

# Crear los splits de train, val y test
create_dataset_splits(original_dataset_dir, output_dataset_dir)

print(f"Datasets creados en: {output_dataset_dir}")


ValueError: With n_samples=0, test_size=None and train_size=0.7, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:

# Función para aplicar Mixup
def mixup_data(x, y, alpha=ALPHA):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# Función de pérdida de Mixup
def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


In [ ]:

# Definir el modelo preentrenado (por ejemplo, ResNet18)
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))  # Ajuste de la última capa

model = model.to(device)

# Definir la función de pérdida (Focal Loss) y el optimizador
criterion = nn.CrossEntropyLoss()  # La Focal Loss puede implementarse de forma manual si se desea
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5, min_lr=1e-6)


In [ ]:

# Función de entrenamiento y validación
def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs, scheduler):
    best_val_loss = float("inf")
    for epoch in range(epochs):
        # Entrenamiento
        model.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            inputs, targets_a, targets_b, lam = mixup_data(inputs, labels)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = mixup_criterion(criterion, outputs, targets_a, targets_b, lam)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Validación
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        # Ajustar la tasa de aprendizaje
        scheduler.step(val_loss)

        # Mostrar resultados del epoch
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        print(f"Epoch {epoch+1}/{epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

        # Guardar el mejor modelo
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_model.pth")
            print("Mejor modelo guardado")

train_and_validate(model, train_loader, val_loader, criterion, optimizer, EPOCHS, scheduler)
